In [1]:
import os
import sys
import json
import datetime
import multiprocessing

In [2]:
class Util:
    """
        This class is used to implement utility functions

        ...

        Attributes
        ----------
            No attributes
            
            
        Methods
        -------
            parse(d)
                parses a string to dictionary
            
            contains(path, key)
                checks key is present in library or not
            
            secTillNow()
                returns total seconds till now
    """
    def parse(d):
        """
            This function was implemented to parse a string to dictionary.
            
            Parameters
            ----------
                d : str
        """
        
        dictionary = dict() 
        # Removes curly braces and splits the pairs into a list 
        pairs = d.strip('{}').split(', ') 
        for i in pairs: 
            pair = i.split(': ') 
            # Other symbols from the key-value pair should be stripped. 
            dictionary[pair[0].strip('\'\'\"\"')] = pair[1].strip('\'\'\"\"') 
        return dictionary

    def contains(path, key):
        """
            This function was implemented to check whether library has the following key.
            
            Parameters
            ----------
                path : str
                key : str
            
            
            Raises
            ------
                UnexpectedError
                    If the check was not performed correctly.    
        """
        
        try: 
            file = open(path, 'rt') 
            lines = file.read().split('\n') 
            for l in lines: 
                if l != '': 
                    dictionary = parse(l) 
                    #print(dictionary.keys) 
                    if list(dictionary.keys())[0] == key:
                        return True
            file.close() 
        except: 
            print("Something unexpected occurred!")

        return False
        
    def secTillNow():
        """
            This function was implemented to return total seconds till now from 00:00.
            
            Parameters
            ----------
                No parameters  
        """
        
        now = datetime.datetime.now()
        midnight = datetime.datetime.combine(now.date(), datetime.time())
        seconds = (now - midnight).seconds
        return seconds


In [4]:
    
class Library:
    """
        This class is used to implement utility functions

        ...

        Attributes
        ----------
            path : str
                stores the location of library file
            
            fileName : str ---> private
                stores the library storage name
            
            
        Methods
        -------
            create(key, value, timeToLive=86400)
                stores value in to the library
                
            read(key)
                returns json object associated with the key
                
            delete(key)
                deletes the data associated with the key
    """
    
    def __init__(self, path = None):
        """
            This constructor was implemented initialize the storage if not present.
            
            Parameters
            ----------
                path : str, optional
                    location of the library file, (default = None, i.e, initializes in cwd)
            
            Raises
            ------
                FileNotFoundError
                    If the directiory doesnot exist.
                
                Other
                    with msg -> Unable to initialize library
        """
        if path == None:
            # initialize location when not provided
            self.path = os.getcwd() # get current working directory
        else:
            #location provided
            self.path = path
        
        self.__fileName = "library.txt"  #library file
        
        try:
            self.path = os.path.join(self.path, self.__fileName)
            with open(self.path, "a+") as file: # a+ is used as if file is not available create one
                file.close()
                
        except FileNotFoundError as fnf_error:
            print("Error : ", fnf_error)
            
        except:
            print('Unable to initialize library')
            
            
    def create(self, key, value, timeToLive = 86400, lock):    # error here 
        """
            This function was implemented store data in library.
            
            Parameters
            ----------
                key : str
                    should contains alpha chars only
                
                value: json object
                    
                timeToLive : int, optional
                    uptill when we can access the data (default= 86400)
            
            Raises
            ------
                ValueError
                    If the value is not a json object.
                
                Other
                    with their respective msg.
        """
        if Util.contains(self.path, key) == False:            
            
            #check for the file size < 1GB
            lock.acquire()
            fileStats = os.stat(self.path)
            fileSize = (fileStats.st_size)/(1024*1024)
            if fileSize >= 1024:
                print("file size is too large")
                lock.release()
                return "Operation Unsuccessfull"
            #else:
                #print(fileSize)
                
            # to combine key and timeToLive property together
            dict = {} 
            
            if type(key) == type("") and len(key) <= 32:
                if key.isalpha(): # checks all chars are (a-z)
                    try: 
                        value = json.loads(value) 
                        # print ("Is valid json? true") 
                    except ValueError as e: 
                        print ("It is not valid json object")
                        lock.release()
                        return   # no need to proceed further
                    
                    if(sys.getsizeof(value) <= (1024*16)): #json object < 16KB
                        dict[key] = value
                        tSec = Util.secTillNow()
                        if timeToLive == 86400:
                            dict['timeToLive'] = timeToLive-1 # by default we can access till 23:59:59
                        else:
                            dict['timeToLive'] = (tSec + timeToLive)
                        try:
                            with open(self.path, "a+") as file:
                                file.write(str(dict)+"\n")
                                file.close()
                                print("The value was appended in library")
                        except:
                            print("Unable to perform create operation")
                    else:
                        print("json object size exceeded")
                else:
                    print("key should be alpha numeric")

            else:
                print("Key size exceeded")
            
        else:
            print("Key is already present")
            
        lock.release()
        
        
    
    def read(self, key, lock):
        """
            This function was implemented read data from library.
            
            Parameters
            ----------
                key : str
                    to check data is in library or not
                    
            Raises
            ------
                Other
                    with their respective msg.
        """
        try: 
            lock.acquire()
            file = open(self.path, 'rt') 
            lines = file.read().split('\n') 
            for l in lines: 
                if l != '': 
                    dictionary = eval(l)
                    #print(dictionary.keys) 
                    if list(dictionary.keys())[0] == key:
                        # check the data is accessible or not
                        if dictionary['timeToLive'] >= Util.secTillNow():
                            lock.release()
                            return dictionary[key]
                        else:
                            return "json object is not accessible"
            file.close() 
            lock.release()
        except: 
            print("Something unexpected occurred!")
        
        # if we are here that means key is not in library
        return "Key Not Found"
    
    def delete(self, key, lock):
        """
            This function was implemented read data from library.
            
            Parameters
            ----------
                key : str
                    to check data is in library or not
                    
            Raises
            ------
                Other
                    with their respective msg.
        """
        try:
            # to check whether the data was deleted or not by default false
            flag = False
            
            # read complete file
            lock.acquire()
            readFile = open(self.path, "r")
            lines = readFile.readlines()
            readFile.close()
            #print(lines)
            
            # write the useful data again to the file
            writeFile = open(self.path, "w")
            for line in lines:
                dictionary = eval(line.strip("\n"))
                if list(dictionary.keys())[0] == key:
                    if dictionary['timeToLive'] < Util.secTillNow():
                        writeFile.write(line)
                    else:
                        flag = True
                        deletedVal = dictionary
                else:
                    writeFile.write(line)
            writeFile.close()
            lock.release()
            if flag:
                print("Value was deleted")
                return deletedVal
            else:
                print("Either the key is not present or key is unaccessible")
                
        except:
            print("Something unexpected occured!")

In [8]:
class Test:
    '''Testing Library class'''
    
    # checking the initialization of class
    
    #l1 = Library()
    #l2 = Library("D:")
    #l3 = Library("xyz")
    
    # create fuction check for different errors 
    
    #l1.create("abc", '{"name": "Bob", "languages": "English"}', 800)
    #l1.create("ab", '{"name": "Bob", "languages": "English"}', 800)
    #l1.create("abc1", '{"name": "Bob", "languages": "English"}', 800)
    #l1.create("a", '{"name": "Bob "languages": "English"}', 800)
    #l1.create("abcd", '{"name": "Job", "languages": "English"}', 800)
    #l1.create("abcdf", '{"name": "Job", "languages": "English"}')
    
    # read function check
    
    #print(l1.read("abcd"))
    #print(l1.read("abcde"))
    #print(Util.secTillNow())
    
    # delete function check
    
    #l1.delete("abcf")
    #l1.create("abcf", '{"name": "cob", "languages": "English"}', 800)

In [ ]:
#TODO1 : Multiprocessing of the library file        <------------------------ Work in progress in this file
#TODO2 : Implement chuck reading and delete using multiprocessing

Implementing independent code----------------------------------

In [75]:
print(Util.secTillNow())

70460


In [9]:
print(Test.__doc__)

Testing Library class


In [35]:
len(d)

2

In [39]:
import datetime
def secTillNow():
        now = datetime.datetime.now()
        midnight = datetime.datetime.combine(now.date(), datetime.time())
        seconds = (now - midnight).seconds
        return seconds

In [40]:
print(secTillNow())

55857


In [42]:
(86400 + 55857)%86400


55857

In [1]:
def parse(d): 
    dictionary = dict() 
    # Removes curly braces and splits the pairs into a list 
    pairs = d.strip('{}').split(', ') 
    for i in pairs: 
        pair = i.split(': ') 
        # Other symbols from the key-value pair should be stripped. 
        dictionary[pair[0].strip('\'\'\"\"')] = pair[1].strip('\'\'\"\"') 
    return dictionary

def contains(key): 
    try: 
        file = open("library.txt", 'rt') 
        lines = file.read().split('\n') 
        for l in lines: 
            if l != '': 
                dictionary = parse(l) 
                #print(dictionary.keys) 
                if list(dictionary.keys())[0] == key:
                    return True
        file.close() 
    except: 
        print("Something unexpected occurred!")

    return False

In [2]:
contains("abc")

True

In [88]:
def parse(d): 
	dictionary = dict() 
	# Removes curly braces and splits the pairs into a list 
	pairs = d.strip('{}').split(', ') 
	for i in pairs: 
		pair = i.split(': ') 
		# Other symbols from the key-value pair should be stripped. 
		dictionary[pair[0].strip('\'\'\"\"')] = pair[1].strip('\'\'\"\"') 
	return dictionary 


try: 
	geeky_file = open('library.txt', 'rt') 
	lines = geeky_file.read().split('\n') 
	for l in lines: 
		if l != '': 
			dictionary = parse(l) 
			print(dictionary) 
	geeky_file.close() 
except: 
	print("Something unexpected occurred!")


{'abcd': "{'name", 'languages': "English'}", 'timeToLive': '61255'}
{'abc': "{'name", 'languages': "English'}", 'timeToLive': '61255'}
{'ab': "{'name", 'languages': "English'}", 'timeToLive': '61255'}


In [68]:
readFile = open("library.txt", "r")
lines = readFile.readlines()
readFile.close()
print(lines)

["{'abcd': {'name': 'Bob', 'languages': 'English'}, 'timeToLive': 67380}\n", "{'abc': {'name': 'Bob', 'languages': 'English'}, 'timeToLive': 67380}\n", "{'ab': {'name': 'Bob', 'languages': 'English'}, 'timeToLive': 67380}\n", "{'abcde': {'name': 'Job', 'languages': 'English'}, 'timeToLive': 69873}\n", "{'abcdf': {'name': 'Job', 'languages': 'English'}, 'timeToLive': 86399}\n"]
